In [2]:

import os 
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt

In [3]:
plays = pd.read_csv('plays.csv')
players = pd.read_csv('players.csv')
playerplay = pd.read_csv('player_play.csv')
df = pd.read_csv('tracking_week_1.csv')

In [7]:
lineset = df[df['event']=='line_set']

In [13]:
lineset

,gameId,playId,nflId,displayName,frameId,frameType,time,jerseyNumber,club,playDirection,x,y,s,a,dis,o,dir,event
42,2022091200,64,35459.0,Kareem Jackson,43,BEFORE_SNAP,2022-09-13 00:16:07.7,22.0,DEN,right,51.880000,29.07,0.34,0.26,0.03,261.81,303.38,line_set
205,2022091200,64,39987.0,Geno Smith,43,BEFORE_SNAP,2022-09-13 00:16:07.7,7.0,SEA,right,36.200000,24.01,1.14,0.01,0.11,87.92,79.90,line_set
368,2022091200,64,41310.0,Gabe Jackson,43,BEFORE_SNAP,2022-09-13 00:16:07.7,66.0,SEA,right,38.440000,22.56,0.12,0.48,0.01,57.01,312.06,line_set
531,2022091200,64,42393.0,Ronald Darby,43,BEFORE_SNAP,2022-09-13 00:16:07.7,23.0,DEN,right,45.840000,14.00,0.91,0.52,0.09,332.02,130.48,line_set
694,2022091200,64,42403.0,Randy Gregory,43,BEFORE_SNAP,2022-09-13 00:16:07.7,5.0,DEN,right,40.590000,18.37,0.29,0.95,0.03,286.94,264.41,line_set
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7103889,2022090800,3696,53678.0,Ben Skowronek,65,BEFORE_SNAP,2022-09-09 03:07:27.1,18.0,LA,left,28.460000,43.94,0.28,0.85,0.02,276.66,191.09,line_set
7104064,2022090800,3696,54488.0,Kaiir Elam,65,BEFORE_SNAP,2022-09-09 03:07:27.1,24.0,BUF,left,25.680000,44.13,0.50,1.05,0.06,76.45,344.78,line_set
7104239,2022090800,3696,54554.0,Terrel Bernard,65,BEFORE_SNAP,2022-09-09 03:07:27.1,43.0,BUF,left,15.400000,34.29,0.82,0.17,0.08,102.10,338.80,line_set
7104414,2022090800,3696,54650.0,Christian Benford,65,BEFORE_SNAP,2022-09-09 03:07:27.1,47.0,BUF,left,16.920000,8.36,1.14,0.43,0.12,77.93,254.67,line_set


In [11]:
motionpositions = players[['nflId','position']][(players['position'] == "WR") | (players['position'] == "RB") | (players['position'] == "TE") | (players['position'] == "CB") | (players['position'] == "OLB") | (players['position'] == "DE") | (players['position'] == "ILB") | (players['position'] == "FS") | (players['position'] == "SS")]

In [14]:
motionplayers = pd.merge(
    lineset,
    motionpositions,
    how = "left",
    left_on = ['nflId'],
    right_on = ['nflId']
)

In [32]:
motionplayers = motionplayers[motionplayers['position'].notna()]
motionplayers = motionplayers[['gameId','playId','nflId','x','y','position']]
motionplayers

,gameId,playId,nflId,x,y,position
0,2022091200,64,35459.0,51.88,29.07,SS
1,2022091200,64,42393.0,45.84,14.00,CB
2,2022091200,64,42403.0,40.59,18.37,OLB
3,2022091200,64,42412.0,39.06,14.49,WR
4,2022091200,64,42826.0,40.69,22.06,DE
...,...,...,...,...,...,...
26818,2022090800,3696,53459.0,25.73,35.49,DE
26819,2022090800,3696,53641.0,14.15,21.12,SS
26820,2022090800,3696,53678.0,28.46,43.94,WR
26821,2022090800,3696,54488.0,25.68,44.13,CB


In [143]:
offensive_players = motionplayers[motionplayers['position'].isin(["WR", "TE", "RB"])].copy()
defensive_players = motionplayers[~motionplayers['position'].isin(["WR", "TE", "RB"])].copy()


In [144]:
offensive_players['defender'] = None

In [145]:
for i in range(len(offensive_players)):
    game = offensive_players.iloc[i]['gameId']
    play = offensive_players.iloc[i]['playId']
    potentialDefenders = defensive_players[(defensive_players['gameId'] == game) & (defensive_players['playId'] == play)]
    closest = 99999999
    closestid = 0
    for j in range(len(potentialDefenders)):
        if abs(offensive_players.iloc[i]['y'] - potentialDefenders.iloc[j]['y']) < closest:
            closest = abs(offensive_players.iloc[i]['y'] - potentialDefenders.iloc[j]['y'])
            closestid = potentialDefenders.iloc[j]['nflId']
    offensive_players.loc[offensive_players.index[i], 'defender'] = closestid

In [146]:
offensive_players

,gameId,playId,nflId,x,y,position,defender
3,2022091200,64,42412.0,39.06,14.49,WR,42393.0
8,2022091200,64,46096.0,31.89,23.92,RB,52706.0
9,2022091200,64,46189.0,34.72,26.46,TE,42929.0
10,2022091200,64,47803.0,37.59,28.49,TE,35459.0
11,2022091200,64,47847.0,38.53,33.35,WR,53438.0
...,...,...,...,...,...,...,...
26811,2022090800,3696,41290.0,27.14,8.13,WR,54650.0
26812,2022090800,3696,43399.0,27.36,37.08,TE,47964.0
26813,2022090800,3696,44881.0,28.20,17.94,WR,46223.0
26815,2022090800,3696,47853.0,31.67,31.85,RB,53459.0


In [147]:
plays[['gameId','playId','pff_manZone']]

,gameId,playId,pff_manZone
0,2022102302,2655,Zone
1,2022091809,3698,Zone
2,2022103004,3146,Zone
3,2022110610,348,Zone
4,2022102700,2799,Man
...,...,...,...
16119,2022110604,1051,Zone
16120,2022103005,3492,Zone
16121,2022092502,2337,Zone
16122,2022091809,719,Zone


In [148]:
offensive_players = pd.merge(
    offensive_players,
    plays[['gameId','playId','pff_manZone']],
    how="left",
    left_on = ['gameId','playId'],
    right_on = ['gameId','playId']
)

In [149]:
offensive_players = offensive_players[offensive_players['pff_manZone']=="Man"]

In [150]:
offensive_players

,gameId,playId,nflId,x,y,position,defender,pff_manZone
25,2022091200,201,40026.0,70.11,18.71,WR,42030.0,Man
26,2022091200,201,42412.0,70.07,39.44,WR,42393.0,Man
27,2022091200,201,46189.0,70.71,34.92,TE,35459.0,Man
28,2022091200,201,47847.0,70.76,10.03,WR,53438.0,Man
29,2022091200,201,47987.0,66.42,27.74,RB,47854.0,Man
...,...,...,...,...,...,...,...,...
9351,2022090800,3190,41290.0,71.53,7.35,WR,52647.0,Man
9352,2022090800,3190,43399.0,72.77,30.59,TE,40166.0,Man
9353,2022090800,3190,44881.0,72.39,16.98,WR,46190.0,Man
9354,2022090800,3190,47853.0,75.90,21.79,RB,46085.0,Man
